In [1]:
import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from matplotlib.pylab import style
style.use('ggplot')    
plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.rcParams['axes.unicode_minus'] = False  

# 导入数据

In [2]:
def merge_df(dt):
    df_resolve_new = pd.read_csv(f'stat{dt}/resolve_stat/gdl_jrs_msg_parse_baseline_2_3.csv',
                             index_col = False)
    df_resolve_init = pd.read_csv('resolve/gdl_jrs_msg_parse_baseline_2_3.csv',index_col = False)
    df_resolve_init['table_name'] = 'gdl_jrs_msg_parse'
    df_resolve_new.drop_duplicates(inplace = True)
    df_resolve_init.drop_duplicates(inplace = True)
    df_resolve = df_resolve_init.append(df_resolve_new).sort_values('dt').reset_index(drop = True)
    df_resolve.drop_duplicates(inplace = True)
    df_resolve.to_csv('resolve/gdl_jrs_msg_parse_baseline_2_3.csv',index = False)
    return df_resolve

In [52]:
def industry_ana(date_start,date_end,df_category_import,important_category):
    with pd.ExcelWriter('result_excel/result_2_3/baseline_2_3_industry.xlsx') as writer:
        for i in important_category:
            df_category_industry = df_category_import[(df_category_import['category_2']==i) & (df_category_import['dt'] > date_start) & (df_category_import['dt'] < date_end)].sort_values(['dt','category_3']).reset_index(drop = True)
            df_category_industry_3 = df_category_industry.groupby(['三级分类含义']).agg('sum')[['total_num','unique_num']].reset_index()
            df_category_industry_3['unique_num_weight'] = df_category_industry_3['unique_num']/df_category_industry_3['unique_num'].sum()
            df_category_industry_3['total_num_weight'] = df_category_industry_3['total_num']/df_category_industry_3['total_num'].sum()
            df_category_industry_3['week'] = date_start
            desc = df_category_industry['二级分类含义'].unique()[0]
            plt.figure(figsize=(15, 5), dpi=80)
            plt.bar(df_category_industry_3['三级分类含义'],df_category_industry_3['total_num'],color = "#87CEFA")
            plt.ylabel(f'{desc}用户各行为的短信条数(万)')
            for a,b in zip(df_category_industry_3['三级分类含义'],df_category_industry_3['total_num']):
                plt.text(a, b-0.3,'%.2f'%round(b/100000,3), ha = 'center',va = 'bottom',fontsize=15)
            plt.savefig(f'result_picture/result_2_3/result_{date_start}_{date_end}/{date_start}_{date_end}_{desc}用户各行为的短信条数', dpi=300)
            plt.close()
            plt.figure(figsize=(15, 8), dpi=80)
            plt.bar(df_category_industry_3['三级分类含义'],df_category_industry_3['unique_num'],color = "#87CEFA")
            plt.ylabel(f'{desc}用户各行为的独立用户数(万)')
            for a,b in zip(df_category_industry_3['三级分类含义'],df_category_industry_3['unique_num']):
                plt.text(a, b-0.3,'%.2f'%round(b/10000,3), ha = 'center',va = 'bottom',fontsize=15)
            plt.savefig(f'result_picture/result_2_3/result_{date_start}_{date_end}/{date_start}_{date_end}_{desc}用户各行为的独立用户数', dpi=300)
            plt.close()
            plt.figure(1,figsize=(20,20), dpi=80)#将画布设定为正方形，则绘制的饼图是正圆
            plt.axes(aspect=1) 
            industry_3 = list(df_category_industry_3['三级分类含义'])
            explodes = []
            l = 1
            m = 0.1
            for index,key in enumerate(industry_3): 
                if float(df_category_industry_3[df_category_industry_3['三级分类含义'] == key]['total_num_weight'])>0.001:
                    explodes.append(0.0)
                else:
                    if explodes[index - 1] == 0.0:
                        l = 1
                    explodes.append(l*m)
                    l = l+2
            plt.title(f'{date_start}_{date_end}_{desc}用户各行为短信条数占比',fontsize = 20,fontweight = 'black',x = 0.5)#绘制标题
            plt.pie(x = df_category_industry_3['total_num'],autopct='%3.1f %%', startangle=90, pctdistance=0.8,explode =explodes)
            plt.legend(labels=df_category_industry_3['三级分类含义'])
            plt.savefig(f'result_picture/result_2_3/result_{date_start}_{date_end}/{date_start}_{date_end}_{desc}用户各行为的短信条数占比', dpi=300)
            plt.close()
            plt.figure(1,figsize=(20,20), dpi=80)#将画布设定为正方形，则绘制的饼图是正圆
            plt.axes(aspect=1) 
            plt.title(f'{date_start}_{date_end}_{desc}用户各行为用户数占比',fontsize = 20,fontweight = 'black',x = 0.5)#绘制标题
            plt.pie(x = df_category_industry_3['unique_num'],autopct='%3.1f %%', startangle=90, pctdistance=0.8,explode = explodes)
            plt.legend(labels=df_category_industry_3['三级分类含义'],)
            plt.savefig(f'result_picture/result_2_3/result_{date_start}_{date_end}/{date_start}_{date_end}_{desc}用户各行为的用户数占比', dpi=300)
            plt.close()
            if os.path.exists('result_excel/result_2_3/baseline_2_3_industry.xlsx') :
                df_category_industry_3_1 = pd.read_excel('result_excel/result_2_3/baseline_2_3_industry.xlsx',sheet_name = f'{desc}用户的短信和独立用户数',index = False)
                df_category_industry_3_1 = df_category_industry_3_1.append(df_category_industry_3)
            else:
                df_category_industry_3_1 = df_category_industry_3
            df_category_industry_3_1.to_excel(writer,sheet_name = f'{desc}用户的短信和独立用户数',index = False)

In [4]:
def industry_all_ana(date_start,date_end,df_category_import):
    df_category_import_week = df_category_import[(df_category_import['dt'] > date_start) & (df_category_import['dt'] < date_end)]
    df_category_import_week1 = df_category_import_week.groupby(['二级分类含义']).agg('sum')[['total_num','unique_num']].reset_index()
    df_category_import_week1['week'] = date_start
    if os.path.exists('result_excel/result_2_3/baseline_2_3_week.xlsx'):
        df_category_import_week2 = pd.read_excel('result_excel/result_2_3/baseline_2_3_week.xlsx',index = False)
        df_category_import_week2 = df_category_import_week2.append(df_category_import_week1)
    else:
        df_category_import_week2 = df_category_import_week1
    with pd.ExcelWriter('result_excel/result_2_3/baseline_2_3_week.xlsx') as writer:
         df_category_import_week2.to_excel(writer,sheet_name = '各机构类型的短信和独立用户数',index = False)
    if not os.path.exists(f'result_picture/result_2_3/result_{date_start}_{date_end}'):
        os.mkdir(f'result_picture/result_2_3/result_{date_start}_{date_end}')
    plt.figure(figsize=(10, 8), dpi=80)
    plt.bar(df_category_import_week1['二级分类含义'],df_category_import_week1['total_num'],color = "#87CEFA")
    plt.ylabel(f'{date_start}_{date_end}各机构类型的短信条数(千万)')
    for a,b in zip(df_category_import_week1['二级分类含义'],df_category_import_week1['total_num']):
        plt.text(a, b-0.3,'%.2f'%round(b/10000000,3), ha = 'center',va = 'bottom',fontsize=15)
    plt.savefig(f'result_picture/result_2_3/result_{date_start}_{date_end}/{date_start}_{date_end}_各行为的短信条数', dpi=300)
    plt.close()
    plt.figure(figsize=(10, 8), dpi=80)
    plt.bar(df_category_import_week1['二级分类含义'],df_category_import_week1['unique_num'],color = "#87CEFA")
    plt.ylabel(f'{date_start}_{date_end}各机构类型的独立用户数(千万)')
    for a,b in zip(df_category_import_week1['二级分类含义'],df_category_import_week1['unique_num']):
        plt.text(a, b-0.3,'%.2f'%round(b/10000000,3), ha = 'center',va = 'bottom',fontsize=15)
    plt.savefig(f'result_picture/result_2_3/result_{date_start}_{date_end}/{date_start}_{date_end}_各行为的用户数', dpi=300)
    plt.close()

In [5]:
def main(date_start,date_final,df_category_import,important_category):
    if not os.path.exists('result_excel/result_2_3'):
        os.mkdir('result_excel/result_2_3')
    if not os.path.exists('result_picture/result_2_3'):
        os.mkdir('result_picture/result_2_3')
    while date_start<date_final:
        print(date_start)
        date_start1 = datetime.strptime(date_start, "%Y-%m-%d")
        date_end = str(date_start1 - timedelta(days=-6))[:-9]
        industry_all_ana(date_start,date_end,df_category_import)
        industry_ana(date_start,date_end,df_category_import,important_category)
        date_start = date_end

In [ ]:
date_start = '2019-03-01'
date_final = '2019-06-08'
df_category = merge_df('20200920')
important_category = ['credit_card','bank_budget','other_budget','loan','business_insurance',
                      'gong_ji_jin','report']
other_import = ['chat','express','games','hotel','onlineshopping','recruit','takeoutfood',
                'taxi','tourism','yys']
df_category_import = df_category[df_category['category_2'].isin(important_category+other_import)]
category_list = pd.read_excel('category2_3.xlsx',index = False)
category_list.rename(columns = {"二级分类名称":"category_2","三级分类名称":"category_3"},inplace = True)
df_category_import =df_category_import.merge(category_list,on = ['category_2','category_3'],how = 'left')
df_category_import = df_category_import[['category_2','二级分类含义','category_3','三级分类含义','total_num','unique_num','dt']]
important_category = important_category+other_import
main(date_start,date_final,df_category_import,important_category+other_import)

2019-03-01
2019-03-07
2019-03-13
2019-03-19
2019-03-25
2019-03-31
2019-04-06
2019-04-12
2019-04-18
2019-04-24
2019-04-30
2019-05-06
2019-05-12
2019-05-18
